Importación y Carga de Datos

In [1]:
# Importaciones
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from imblearn.over_sampling import SMOTE
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from keras_tuner import Hyperband
import tensorflow as tf
from scikeras.wrappers import KerasClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score

df = pd.read_csv('Telco Churn dataset 2.csv')

Using TensorFlow backend


Preprocesamiento

In [2]:
# 1. Preprocesamiento
# a. Crear la columna 'Churn' y asignar 1 si 'Churn' es 'Yes', de lo contrario 0
df['Churn'] = df['Churn'].apply(lambda x: 1 if x == 'Yes' else 0)

# b. Eliminar columnas con más del 50% de datos faltantes
threshold = int(0.5 * len(df))
df = df.dropna(thresh=threshold, axis=1)

# c. Reemplazar valores atípicos por la media
for col in df.select_dtypes(include=['float64', 'int64']):
    if col != 'Churn':  # Asegurarse de no modificar la columna 'Churn'
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        df[col] = df[col].apply(lambda x: df[col].mean() if (x < (Q1 - 1.5 * IQR)) or (x > (Q3 + 1.5 * IQR)) else x)

# d. Convertir variables categóricas a numéricas y llenar valores faltantes
for column in df.columns:
    if df[column].dtype == 'object' and column != 'Churn':
        df[column].fillna(df[column].mode()[0], inplace=True)
        le = LabelEncoder()
        df[column] = le.fit_transform(df[column])
    elif df[column].dtype in ['int64', 'float64']:
        df[column].fillna(df[column].median(), inplace=True)

# e. Normalización
cols_to_scale = df.columns.tolist()
cols_to_scale.remove('Churn')
scaler = StandardScaler()
df[cols_to_scale] = scaler.fit_transform(df[cols_to_scale])

# f. Asegurarse de que 'Churn' sea int
df['Churn'] = df['Churn'].astype(int)


Selección de características - Gradient Boosting

In [3]:
# 2. Selección de características
xgb_for_feature_selection = XGBClassifier(
    objective='binary:logistic', 
    random_state=42, 
    use_label_encoder=False, 
    eval_metric='logloss'
)
xgb_for_feature_selection.fit(df.drop('Churn', axis=1), df['Churn'])
threshold = 0.01  
selected_features = df.drop('Churn', axis=1).columns[(xgb_for_feature_selection.feature_importances_ > threshold)].tolist()
print("Características seleccionadas:", selected_features)

Características seleccionadas: ['customerID', 'SeniorCitizen', 'tenure', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod', 'InternationalPlan', 'VoiceMailPlan', 'NumbervMailMessages', 'TotalDayMinutes', 'TotalEveMinutes', 'TotalNightMinutes', 'TotalIntlMinutes', 'TotalIntlCalls', 'CustomerServiceCalls']


Balanceo con SMOTE

In [4]:
# 3. Balanceo con SMOTE
X = df[selected_features]
y = df['Churn']
smote = SMOTE(random_state=42)
X_smote, y_smote = smote.fit_resample(X, y)

División de Conjunto

In [5]:
# 4. División de Conjunto
X_train, X_test, y_train, y_test = train_test_split(X_smote, y_smote, test_size=0.2, random_state=42)

Preparación de Datos para RNN

In [6]:
# 5. Preparación de Datos para RNN
X_train_rnn = np.reshape(X_train.values, (X_train.shape[0], 1, X_train.shape[1]))
X_test_rnn = np.reshape(X_test.values, (X_test.shape[0], 1, X_test.shape[1]))

Entrenamiento de RNN

In [7]:
# 6. Entrenamiento de RNN con búsqueda de hiperparámetros usando Keras Tuner
def build_rnn_model(hp):
    model = Sequential()
    model.add(LSTM(units=hp.Int('units', min_value=8, max_value=64, step=8), 
                   input_shape=(X_train_rnn.shape[1], X_train_rnn.shape[2]), 
                   return_sequences=True))
    model.add(LSTM(units=hp.Int('units', min_value=8, max_value=64, step=8), return_sequences=False))
    model.add(Dense(1, activation='sigmoid'))
    optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=hp.Choice('learning_rate', values=[0.001, 0.01, 0.1]))
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

tuner_rnn = Hyperband(
    build_rnn_model,
    objective='val_accuracy',
    max_epochs=100,
    hyperband_iterations=2,
    directory='my_dir',
    project_name='keras_tuner_rnn_example'
)

callbacks_rnn = [
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10),
    tf.keras.callbacks.ModelCheckpoint('best_rnn_model.h5', save_best_only=True)
]

tuner_rnn.search(X_train_rnn, y_train, epochs=100, validation_split=0.2, callbacks=callbacks_rnn)

best_hps_rnn = tuner_rnn.get_best_hyperparameters(num_trials=1)[0]

print(f"""
Los mejores hiperparámetros encontrados:
Número de unidades: {best_hps_rnn.get('units')}
Tasa de aprendizaje: {best_hps_rnn.get('learning_rate')}
""")

# Obtener el mejor modelo RNN
best_rnn_model = tuner_rnn.get_best_models(num_models=1)[0]

Trial 24 Complete [00h 00m 16s]
val_accuracy: 0.8870614171028137

Best val_accuracy So Far: 0.8980262875556946
Total elapsed time: 00h 05m 56s

Los mejores hiperparámetros encontrados:
Número de unidades: 40
Tasa de aprendizaje: 0.01



Selección de Hiperparámetros

In [8]:
# 7. Evaluación del modelo con los mejores hiperparámetros
loss_rnn, accuracy_rnn = best_rnn_model.evaluate(X_test_rnn, y_test)
print(f'Loss: {loss_rnn}, Accuracy: {accuracy_rnn}')

# Obtener las predicciones en forma de clases
y_pred_rnn = (best_rnn_model.predict(X_test_rnn) > 0.5).astype("int32")

# Calcular la matriz de confusión, y generar el reporte de clasificación
conf_matrix_rnn = confusion_matrix(y_test, y_pred_rnn)
report_rnn = classification_report(y_test, y_pred_rnn)

# Imprimir las métricas
print(conf_matrix_rnn)
print("Reporte de clasificación:\n", report_rnn)

36/36 [==============================] - 5s 43ms/step - loss: 0.2595 - accuracy: 0.8860
Loss: 0.2594653069972992, Accuracy: 0.8859649300575256
36/36 [==============================] - 2s 10ms/step
[[513  62]
 [ 68 497]]
Reporte de clasificación:
               precision    recall  f1-score   support

           0       0.88      0.89      0.89       575
           1       0.89      0.88      0.88       565

    accuracy                           0.89      1140
   macro avg       0.89      0.89      0.89      1140
weighted avg       0.89      0.89      0.89      1140

